In [126]:
# ── Imports
import os, json, math, datetime as dt
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import Point
from libpysal.weights import Queen, W
from dotenv import load_dotenv
import urllib3

# ── Load secrets (ACLED + Google)
load_dotenv()  # expects .env in your working dir
ACLED_USER = os.getenv("ACLED_USER")
ACLED_PASS = os.getenv("ACLED_PASS")
assert ACLED_USER and ACLED_PASS, "Set ACLED_USER and ACLED_PASS in your .env"

# ── Helper date anchors
TODAY = dt.date.today()
FROM_30 = TODAY - dt.timedelta(days=30)
FROM_90 = TODAY - dt.timedelta(days=90)
FROM_60PREV = TODAY - dt.timedelta(days=60)
TO_30PREV = TODAY - dt.timedelta(days=30)


In [127]:
# suppress the InsecureRequestWarning when disabling SSL verification
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_acled_token(username: str, password: str, verify: bool = True) -> str:
    url = "https://acleddata.com/oauth/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json",
    }
    data = {
        "username": username,
        "password": password,
        "grant_type": "password",
        "client_id": "acled",
    }
    # pass verify to allow skipping SSL verification for servers with self-signed certs
    # NOTE: verify=False is insecure and should be avoided in production
    r = requests.post(url, headers=headers, data=data, timeout=60, verify=verify)
    r.raise_for_status()
    return r.json()["access_token"]

# Use verify=False only to work around self-signed certificate in the chain (development/testing)
ACLED_TOKEN = get_acled_token(ACLED_USER, ACLED_PASS, verify=False)


In [128]:
def acled_fetch(params: dict, token: str, max_pages: int = 200, limit: int = 5000, verify: bool = True) -> pd.DataFrame:
    """
    Fetch ACLED data with given params using bearer token.
    IMPORTANT: For date ranges, pass:
      params["event_date"] = "YYYY-MM-DD|YYYY-MM-DD"
      params["event_date_where"] = "BETWEEN"
    Do NOT expand into multiple event_date keys; ACLED treats repeated keys as OR which inflates results.
    """
    base = "https://acleddata.com/api/acled/read"
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json",
    }

    frames = []
    for page in range(1, max_pages + 1):
        q = {"_format": "json", "page": page, "limit": limit}
        q.update(params)
        r = requests.get(base, headers=headers, params=q, timeout=120, verify=verify)
        r.raise_for_status()
        js = r.json()
        data = js.get("data", [])
        if not data:
            break
        frames.append(pd.DataFrame(data))
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()


In [129]:
# Quick count helper to probe API without downloading everything
# Returns (count, total_count, messages, data_query_restrictions)
def acled_quick_probe(params: dict, token: str, verify: bool = True):
    base = "https://acleddata.com/api/acled/read"
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    q = {"_format": "json", "page": 1, "limit": 1}
    q.update(params)
    r = requests.get(base, headers=headers, params=q, timeout=60, verify=verify)
    r.raise_for_status()
    js = r.json()
    return (
        js.get("count", 0),
        js.get("total_count", 0),
        js.get("messages", []),
        js.get("data_query_restrictions", {}),
    )


In [130]:
FIELDS = "event_id_cnty|event_date|year|event_type|country|admin1|admin2|latitude|longitude"
violent_types = {"Violence against civilians","Battles","Explosions/Remote violence"}

# Respect account recency cap (e.g., access limited to data up to a specific date)
_, _, _, _restr0 = acled_quick_probe({}, ACLED_TOKEN, verify=False)
_allowed_end_str = None
if isinstance(_restr0, dict):
    _dre = _restr0.get("date_recency") or {}
    _allowed_end_str = _dre.get("date")

if _allowed_end_str:
    try:
        _allowed_end = dt.date.fromisoformat(_allowed_end_str)
    except Exception:
        _allowed_end = TODAY
else:
    _allowed_end = TODAY

if _allowed_end < TODAY:
    print(f"Recency cap in effect: ending at {_allowed_end} (was today={TODAY})")

# Anchor all ranges to the allowed end date
TODAY = _allowed_end
FROM_30 = TODAY - dt.timedelta(days=30)
FROM_90 = TODAY - dt.timedelta(days=90)
FROM_60PREV = TODAY - dt.timedelta(days=60)
TO_30PREV = TODAY - dt.timedelta(days=30)

# Build primary (ISO) and fallback (country) parameter sets using the capped dates
params_90_iso = {
    "iso": 484,
    "event_date": f"{FROM_90}|{TODAY}",
    "event_date_where": "BETWEEN",
    "fields": FIELDS,
}
params_prev_iso = {
    "iso": 484,
    "event_date": f"{FROM_60PREV}|{TO_30PREV}",
    "event_date_where": "BETWEEN",
    "fields": FIELDS,
}
params_90_country = {
    "country": "Mexico",
    "event_date": f"{FROM_90}|{TODAY}",
    "event_date_where": "BETWEEN",
    "fields": FIELDS,
}
params_prev_country = {
    "country": "Mexico",
    "event_date": f"{FROM_60PREV}|{TO_30PREV}",
    "event_date_where": "BETWEEN",
    "fields": FIELDS,
}

# Probe ISO first
c1, t1, m1, r1 = acled_quick_probe(params_90_iso, ACLED_TOKEN, verify=False)
if t1 == 0:
    # fallback to country
    c1, t1, m1, r1 = acled_quick_probe(params_90_country, ACLED_TOKEN, verify=False)
    active_90 = params_90_country
else:
    active_90 = params_90_iso

c2, t2, m2, r2 = acled_quick_probe(params_prev_iso, ACLED_TOKEN, verify=False)
if t2 == 0:
    c2, t2, m2, r2 = acled_quick_probe(params_prev_country, ACLED_TOKEN, verify=False)
    active_prev = params_prev_country
else:
    active_prev = params_prev_iso

print("Probe 90d total_count:", t1, "messages:", m1)
print("Probe prev total_count:", t2, "messages:", m2)

# Fetch using whichever probe returned data
acled_90 = acled_fetch(active_90, ACLED_TOKEN, verify=False)
acled_prev = acled_fetch(active_prev, ACLED_TOKEN, verify=False)

# Safety: enforce country text and dedupe by event_id_cnty
for name, df in (("acled_90", acled_90), ("acled_prev", acled_prev)):
    if not df.empty:
        if "country" in df.columns:
            df = df[df["country"] == "Mexico"].copy()
        if "event_id_cnty" in df.columns:
            df = df.drop_duplicates(subset=["event_id_cnty"]).reset_index(drop=True)
    if name == "acled_90":
        acled_90 = df
    else:
        acled_prev = df

# Filter violent events, handle empty DataFrames
if "event_type" in acled_90.columns:
    acled_90v = acled_90[acled_90["event_type"].isin(violent_types)].copy()
else:
    acled_90v = pd.DataFrame(columns=acled_90.columns)

if "event_type" in acled_prev.columns:
    acled_prevv = acled_prev[acled_prev["event_type"].isin(violent_types)].copy()
else:
    acled_prevv = pd.DataFrame(columns=acled_prev.columns)

# Normalize admin name quirks (extend as needed)
def fix_admin_names(df: pd.DataFrame) -> pd.DataFrame:
    rep = {
        "Michoacan de Ocampo": "Michoacán",
        "Queretaro": "Querétaro",
        "Nuevo Leon": "Nuevo León",
        "San Luis Potosi": "San Luis Potosí",
        "Yucatan": "Yucatán",
        "Mexico": "México",
    }
    df = df.copy()
    if "admin1" in df.columns:
        df["admin1"] = df["admin1"].replace(rep)
    return df

acled_90v = fix_admin_names(acled_90v)
acled_prevv = fix_admin_names(acled_prevv)


Recency cap in effect: ending at 2024-10-22 (was today=2025-10-22)
Probe 90d total_count: 4037 messages: []
Probe prev total_count: 1397 messages: []
Probe 90d total_count: 4037 messages: []
Probe prev total_count: 1397 messages: []


In [131]:
acled_90v

,event_id_cnty,event_date,year,event_type,country,admin1,admin2,latitude,longitude
0,MEX90465,2024-07-25,2024,Violence against civilians,Mexico,Jalisco,Lagos de Moreno,21.3564,-101.9292
1,MEX90483,2024-07-24,2024,Battles,Mexico,Guanajuato,Celaya,20.5215,-100.8140
2,MEX90486,2024-07-24,2024,Battles,Mexico,Michoacán,Arteaga,18.2936,-101.9031
3,MEX90489,2024-07-24,2024,Battles,Mexico,Veracruz de Ignacio de la Llave,Tezonapa,18.6068,-96.6875
6,MEX90502,2024-07-26,2024,Violence against civilians,Mexico,Michoacán,Los Reyes,19.5882,-102.4697
...,...,...,...,...,...,...,...,...,...
4031,MEX94768,2024-10-22,2024,Violence against civilians,Mexico,Guanajuato,Leon,21.1220,-101.6832
4032,MEX94769,2024-10-22,2024,Violence against civilians,Mexico,Guanajuato,Leon,21.1220,-101.6832
4034,MEX99159,2024-10-08,2024,Violence against civilians,Mexico,Sinaloa,Culiacan,24.8088,-107.3940
4035,MEX99999,2024-10-16,2024,Violence against civilians,Mexico,Guanajuato,Apaseo el Alto,20.5131,-100.5599


In [132]:
# Build GeoDataFrames of ACLED points from latitude/longitude
from shapely.geometry import Point as _Point


def _to_points(df: pd.DataFrame) -> gpd.GeoDataFrame:
    if df.empty:
        return gpd.GeoDataFrame(df.copy(), geometry=[], crs="EPSG:4326")
    # Ensure numeric
    d = df.copy()
    for col in ("latitude", "longitude"):
        if col in d.columns:
            d[col] = pd.to_numeric(d[col], errors="coerce")
    # Drop missing coords
    d = d.dropna(subset=["latitude", "longitude"]).copy()
    if d.empty:
        return gpd.GeoDataFrame(d, geometry=[], crs="EPSG:4326")
    geom = [_Point(lon, lat) for lat, lon in zip(d["latitude"], d["longitude"]) ]
    gdf = gpd.GeoDataFrame(d, geometry=geom, crs="EPSG:4326")
    return gdf

acled_90_g = _to_points(acled_90)
acled_prev_g = _to_points(acled_prev)
acled_90v_g = _to_points(acled_90v)
acled_prevv_g = _to_points(acled_prevv)

print("Points built:")
print("acled_90_g:", acled_90_g.shape, "CRS:", acled_90_g.crs)
print("acled_prev_g:", acled_prev_g.shape, "CRS:", acled_prev_g.crs)
print("acled_90v_g:", acled_90v_g.shape, "CRS:", acled_90v_g.crs)
print("acled_prevv_g:", acled_prevv_g.shape, "CRS:", acled_prevv_g.crs)

# Preview a few coordinates
if not acled_90_g.empty:
    print(acled_90_g[["event_date", "event_type", "latitude", "longitude"]].head())


Points built:
acled_90_g: (4037, 10) CRS: EPSG:4326
acled_prev_g: (1397, 10) CRS: EPSG:4326
acled_90v_g: (1921, 10) CRS: EPSG:4326
acled_prevv_g: (613, 10) CRS: EPSG:4326
   event_date                  event_type  latitude  longitude
0  2024-07-25  Violence against civilians   21.3564  -101.9292
1  2024-07-24                     Battles   20.5215  -100.8140
2  2024-07-24                     Battles   18.2936  -101.9031
3  2024-07-24                     Battles   18.6068   -96.6875
4  2024-07-25      Strategic developments   25.8080  -100.3226


## INEGI administrative polygons (2024)

We'll cache the official Marco Geoestadístico 2024 national shapefile locally, extract it, and auto-load the state and municipality layers into GeoDataFrames for spatial joins with ACLED points.

Source ZIP:
- https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/794551132173_s.zip

In [133]:
# Download and extract INEGI 2024 national shapefile (cached)
import os, zipfile, pathlib, shutil
from urllib.parse import urlparse
import requests

INEGI_ZIP_URL = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/794551132173_s.zip"
DATA_DIR = pathlib.Path.cwd() / "data"
CACHE_DIR = DATA_DIR / "inegi_2024"
ZIP_PATH = CACHE_DIR / pathlib.Path(urlparse(INEGI_ZIP_URL).path).name
EXTRACT_DIR = CACHE_DIR / "extracted"
FORCE_REFRESH = False  # set True to re-download/re-extract

DATA_DIR.mkdir(exist_ok=True)
CACHE_DIR.mkdir(exist_ok=True)

# Download if missing or forced
if FORCE_REFRESH or not ZIP_PATH.exists():
    print(f"Downloading INEGI ZIP to {ZIP_PATH} ...")
    with requests.get(INEGI_ZIP_URL, stream=True, timeout=300) as r:
        r.raise_for_status()
        with open(ZIP_PATH, "wb") as f:
            for chunk in r.iter_content(chunk_size=1 << 20):
                if chunk:
                    f.write(chunk)
    print("Download complete.")
else:
    print(f"ZIP already present: {ZIP_PATH}")

# Extract if missing or forced
if FORCE_REFRESH or not EXTRACT_DIR.exists():
    if EXTRACT_DIR.exists():
        shutil.rmtree(EXTRACT_DIR)
    EXTRACT_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Extracting to {EXTRACT_DIR} ...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
        zf.extractall(EXTRACT_DIR)
    print("Extraction complete.")
else:
    print(f"Already extracted: {EXTRACT_DIR}")

SSLError: HTTPSConnectionPool(host='www.inegi.org.mx', port=443): Max retries exceeded with url: /contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/794551132173_s.zip (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1010)')))

In [ ]:
# Auto-detect and load state (AGEE) and municipality (AGEM) layers
import re

# Helper to find shapefiles by typical name patterns
PAT_STATE = re.compile(r"AGEE|ENTIDADES|ESTATAL|ENTIDAD", re.IGNORECASE)
PAT_MUNI = re.compile(r"AGEM|MUNICIP", re.IGNORECASE)

shp_files = [p for p in EXTRACT_DIR.rglob("*.shp")]
print(f"Found {len(shp_files)} shapefile(s)")

state_shp = None
muni_shp = None
for p in shp_files:
    name = p.name
    if state_shp is None and PAT_STATE.search(name):
        state_shp = p
    if muni_shp is None and PAT_MUNI.search(name):
        muni_shp = p

print("State shp:", state_shp)
print("Muni shp:", muni_shp)

# Load with geopandas
states_gdf = gpd.read_file(state_shp) if state_shp else gpd.GeoDataFrame()
muni_gdf = gpd.read_file(muni_shp) if muni_shp else gpd.GeoDataFrame()

# Standardize CRS to EPSG:4326
if not states_gdf.empty and states_gdf.crs is not None:
    states_gdf = states_gdf.to_crs(4326)
if not muni_gdf.empty and muni_gdf.crs is not None:
    muni_gdf = muni_gdf.to_crs(4326)

# Try to standardize common columns
# We'll create canonical columns: state_name, state_code, muni_name, muni_code
states_gdf = states_gdf.copy()
muni_gdf = muni_gdf.copy()

# States: attempt common INEGI fields (NOMGEO, CVE_ENT)
if not states_gdf.empty:
    if "NOMGEO" in states_gdf.columns:
        states_gdf["state_name"] = states_gdf["NOMGEO"]
    elif "NOMBRE" in states_gdf.columns:
        states_gdf["state_name"] = states_gdf["NOMBRE"]
    else:
        # fallback: first string column
        str_cols = [c for c in states_gdf.columns if states_gdf[c].dtype == object]
        states_gdf["state_name"] = states_gdf[str_cols[0]] if str_cols else ""
    if "CVE_ENT" in states_gdf.columns:
        states_gdf["state_code"] = states_gdf["CVE_ENT"].astype(str)
    else:
        # try CVEGEO or similar
        cand = next((c for c in states_gdf.columns if c.upper().startswith("CVE")), None)
        states_gdf["state_code"] = states_gdf[cand].astype(str) if cand else ""

# Municipalities: typical INEGI fields: NOMGEO, CVE_ENT, CVE_MUN, CVEGEO
if not muni_gdf.empty:
    if "NOMGEO" in muni_gdf.columns:
        muni_gdf["muni_name"] = muni_gdf["NOMGEO"]
    elif "NOM_MUN" in muni_gdf.columns:
        muni_gdf["muni_name"] = muni_gdf["NOM_MUN"]
    else:
        str_cols = [c for c in muni_gdf.columns if muni_gdf[c].dtype == object]
        muni_gdf["muni_name"] = muni_gdf[str_cols[0]] if str_cols else ""

    # Build muni_code as CVE_ENT + CVE_MUN if possible, else CVEGEO
    if {"CVE_ENT", "CVE_MUN"}.issubset(muni_gdf.columns):
        muni_gdf["muni_code"] = (
            muni_gdf["CVE_ENT"].astype(str).str.zfill(2)
            + muni_gdf["CVE_MUN"].astype(str).str.zfill(3)
        )
    elif "CVEGEO" in muni_gdf.columns:
        muni_gdf["muni_code"] = muni_gdf["CVEGEO"].astype(str)
    else:
        cand = next((c for c in muni_gdf.columns if c.upper().startswith("CVE")), None)
        muni_gdf["muni_code"] = muni_gdf[cand].astype(str) if cand else ""

print("States:", states_gdf.shape, "CRS:", states_gdf.crs)
print("Municipalities:", muni_gdf.shape, "CRS:", muni_gdf.crs)

# Quick integrity hints
if not states_gdf.empty:
    print("State sample:")
    print(states_gdf[["state_code", "state_name"]].head())
if not muni_gdf.empty:
    print("Municipality sample:")
    print(muni_gdf[["muni_code", "muni_name"]].head())

In [ ]:
# Optional: basic integrity checks
if not states_gdf.empty and not muni_gdf.empty:
    # Mexico has 32 federative entities
    n_states = len(states_gdf)
    print(f"States count: {n_states} (expected ~32)")

    # Municipalities expected roughly ~2500-2505 depending on year/version
    n_muni = len(muni_gdf)
    print(f"Municipalities count: {n_muni} (expected ~2500)")

else:
    print("Warning: One or both layers empty. Check patterns or ZIP content.")

In [ ]:
# Spatial join: tag ACLED points with municipality/state and summarize counts
# Use gpd.sjoin for compatibility across GeoPandas versions

# Ensure CRS alignment
for g in (acled_90v_g, acled_prevv_g, muni_gdf, states_gdf):
    if hasattr(g, "crs") and g.crs and g.crs.to_epsg() != 4326:
        g.to_crs(4326, inplace=True)

# Join violent events (last 90 days)
if not acled_90v_g.empty and not muni_gdf.empty:
    acled_90v_muni = gpd.sjoin(acled_90v_g, muni_gdf[["muni_code", "muni_name", "geometry"]], how="left", predicate="intersects")
    muni_counts_90v = (
        acled_90v_muni.groupby(["muni_code", "muni_name"], dropna=False)["event_id_cnty"].count().reset_index(name="events_90v")
    )
    print("Municipality counts (violent, last 90 days):", muni_counts_90v.shape)
    print(muni_counts_90v.head())
else:
    muni_counts_90v = pd.DataFrame()
    print("Skipping muni join: missing inputs")

# Join violent events (previous 30-day window)
if not acled_prevv_g.empty and not muni_gdf.empty:
    acled_prevv_muni = gpd.sjoin(acled_prevv_g, muni_gdf[["muni_code", "muni_name", "geometry"]], how="left", predicate="intersects")
    muni_counts_prevv = (
        acled_prevv_muni.groupby(["muni_code", "muni_name"], dropna=False)["event_id_cnty"].count().reset_index(name="events_prevv")
    )
    print("Municipality counts (violent, previous 30 days):", muni_counts_prevv.shape)
    print(muni_counts_prevv.head())
else:
    muni_counts_prevv = pd.DataFrame()
    print("Skipping muni join (prev): missing inputs")

# Optional: state-level aggregation using admin1 or polygon join
if not acled_90v_g.empty and not states_gdf.empty:
    acled_90v_state = gpd.sjoin(acled_90v_g, states_gdf[["state_code", "state_name", "geometry"]], how="left", predicate="intersects")
    state_counts_90v = (
        acled_90v_state.groupby(["state_code", "state_name"], dropna=False)["event_id_cnty"].count().reset_index(name="events_90v")
    )
    print("State counts (violent, last 90 days):", state_counts_90v.shape)
    print(state_counts_90v.head())
else:
    state_counts_90v = pd.DataFrame()
    print("Skipping state join: missing inputs")